# **Qwen-image-Edit 2509 (ComfyUI)**
- Run the cell below to get a link (e.g. https://localhost:8188/) which you can use to launch the comfyUI interface.
- You can get the .json workflows here: https://github.com/Isi-dev/Google-Colab_Notebooks/tree/main/ComfyUI/Qwen_Image_Edit_Plus
- You can find models here: https://huggingface.co/QuantStack/Qwen-Image-Edit-2509-GGUF/tree/main
- **To use a lora, put its huggingface or civitai download link in the `lora_download_url` textbox, select the `download_lora` checkbox, and if using civitai, input your civitai token before running the code to `Prepare Environment`. Remember to describe the main subject of the image and include the trigger words for the LoRA in the prompt. You can watch this video to learn how to get and use LoRAs from huggingface and civitai, and how to create your civitai token: https://youtu.be/49NkIV_QpBM**
- Github project page: https://github.com/QwenLM/Qwen-Image
- Notebook source: https://github.com/Isi-dev/Google-Colab_Notebooks
- Premium notebooks I highly recommend: https://isinse.gumroad.com/
- Google Colab Youtube Playlist: https://www.youtube.com/playlist?list=PLdi1sS5pbSYeA470Sb1wARR4OieCBIqMv
- Even $1 helps support my work: https://buymeacoffee.com/isiomo



In [ ]:
# @markdown # 💥Prepare Environment

!pip install torch==2.8.0 torchvision==0.23.0

using_T4_GPU = True # @param {type:"boolean"}
include_manager = False # @param {type:"boolean"}

%cd /content
from IPython.display import clear_output
clear_output()
!pip install -q torchsde einops diffusers accelerate xformers==0.0.32.post1 triton==3.4 sageattention
!pip install av spandrel albumentations onnx opencv-python onnxruntime
!pip install onnxruntime-gpu -y
!git clone https://github.com/comfyanonymous/ComfyUI
# !git clone --branch ComfyUI_v0.3.52 https://github.com/Isi-dev/ComfyUI
!pip install -r /content/ComfyUI/requirements.txt
clear_output()


%cd /content/ComfyUI/custom_nodes
# !git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git
if include_manager:
    !git clone https://github.com/ltdrdata/ComfyUI-Manager
!git clone --branch forQwen https://github.com/Isi-dev/ComfyUI_GGUF.git
!git clone https://github.com/Isi-dev/ComfyUI_KJNodes.git
!git clone https://github.com/Isi-dev/ComfyUI_DeleteModelPassthrough.git
!git clone https://github.com/Isi-dev/comfyui_controlnet_aux
%cd /content/ComfyUI/custom_nodes/ComfyUI_GGUF
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI_KJNodes
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI_DeleteModelPassthrough
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/comfyui_controlnet_aux
!pip install -r requirements.txt
if include_manager:
    %cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
    !pip install -r requirements.txt

clear_output()


%cd /content/ComfyUI


import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import subprocess
import sys
from pathlib import Path
# sys.path.insert(0, '/content/ComfyUI')

def install_apt_packages():
    packages = ['aria2']

    try:
        # Run apt install silently (using -qq)
        subprocess.run(
            ['apt-get', '-y', 'install', '-qq'] + packages,
            check=True,
            capture_output=True
        )
        print("✓ apt packages installed")
    except subprocess.CalledProcessError as e:
        print(f"✗ Error installing apt packages: {e.stderr.decode().strip() or 'Unknown error'}")


print("Installing apt packages...")
install_apt_packages()

def download_with_aria2c(link, folder="/content/ComfyUI/models/loras"):
    import os

    filename = link.split("/")[-1]
    command = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d {folder} -o {filename}"

    print("Executing download command:")
    print(command)

    os.makedirs(folder, exist_ok=True)
    get_ipython().system(command)

    return filename



def download_civitai_model(civitai_link, civitai_token, folder="/content/ComfyUI/models/loras"):
    import os
    import time

    os.makedirs(folder, exist_ok=True)

    try:
        model_id = civitai_link.split("/models/")[1].split("?")[0]
    except IndexError:
        raise ValueError("Invalid Civitai URL format. Please use a link like: https://civitai.com/api/download/models/1523247?...")

    civitai_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor"
    if civitai_token:
        civitai_url += f"&token={civitai_token}"

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"model_{timestamp}.safetensors"

    full_path = os.path.join(folder, filename)

    download_command = f"wget --max-redirect=10 --show-progress \"{civitai_url}\" -O \"{full_path}\""
    print("Downloading from Civitai...")

    os.system(download_command)

    local_path = os.path.join(folder, filename)
    if os.path.exists(local_path) and os.path.getsize(local_path) > 0:
        print(f"LoRA downloaded successfully: {local_path}")
    else:
        print(f"❌ LoRA download failed or file is empty: {local_path}")

    return filename

def download_lora(link, folder="/content/ComfyUI/models/loras", civitai_token=None):
    """
    Download a model file, automatically detecting if it's a Civitai link or huggingface download.

    Args:
        link: The download URL (either huggingface or Civitai)
        folder: Destination folder for the download
        civitai_token: Optional token for Civitai downloads (required if link is from Civitai)

    Returns:
        The filename of the downloaded model
    """
    if "civitai.com" in link.lower():
        if not civitai_token:
            raise ValueError("Civitai token is required for Civitai downloads")
        return download_civitai_model(link, civitai_token, folder)
    else:
        return download_with_aria2c(link, folder)

def model_download(url: str, dest_dir: str, filename: str = None, silent: bool = True) -> bool:
    """
    Colab-optimized download with aria2c

    Args:
        url: Download URL
        dest_dir: Target directory (will be created if needed)
        filename: Optional output filename (defaults to URL filename)
        silent: If True, suppresses all output (except errors)

    Returns:
        bool: True if successful, False if failed
    """
    try:
        # Create destination directory
        Path(dest_dir).mkdir(parents=True, exist_ok=True)

        # Set filename if not specified
        if filename is None:
            filename = url.split('/')[-1].split('?')[0]  # Remove URL parameters

        # Build command
        cmd = [
            'aria2c',
            '--console-log-level=error',
            '-c', '-x', '16', '-s', '16', '-k', '1M',
            '-d', dest_dir,
            '-o', filename,
            url
        ]

        # Add silent flags if requested
        if silent:
            cmd.extend(['--summary-interval=0', '--quiet'])
            print(f"Downloading {filename}...", end=' ', flush=True)

        # Run download
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)

        if silent:
            print("Done!")
        else:
            print(f"Downloaded {filename} to {dest_dir}")
        return filename

    except subprocess.CalledProcessError as e:
        error = e.stderr.strip() or "Unknown error"
        print(f"\nError downloading {filename}: {error}")
        return False
    except Exception as e:
        print(f"\nError: {str(e)}")
        return False

# qwen_model_download_url = "https://huggingface.co/city96/Qwen-Image-gguf/resolve/main/qwen-image-Q4_K_M.gguf"# @param {"type":"string"}
# also_download_qwen_image_edit = False # @param {type:"boolean"}
qwen_edit_model_download_url = "https://huggingface.co/QuantStack/Qwen-Image-Edit-2509-GGUF/resolve/main/Qwen-Image-Edit-2509-Q4_K_M.gguf"# @param {"type":"string"}
qwen_text_encoder_download_url = "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/text_encoders/qwen_2.5_vl_7b_fp8_scaled.safetensors"# @param {"type":"string"}
qwen_vae_download_url = "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/vae/qwen_image_vae.safetensors"# @param {"type":"string"}
qwen_speedup_lora_download_url = "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Lightning-4steps-V1.0-bf16.safetensors"# @param {"type":"string"}
# qwen_instantX_controlnet_download_url = "https://huggingface.co/Comfy-Org/Qwen-Image-InstantX-ControlNets/resolve/main/split_files/controlnet/Qwen-Image-InstantX-ControlNet-Union.safetensors"# @param {"type":"string"}


download_loRA = False # @param {type:"boolean"}
qwen_lora_download_url = ""# @param {"type":"string"}
token_if_civitai_url = ""# @param {"type":"string"}



lora = None
if download_loRA:
    lora = download_lora(qwen_lora_download_url, civitai_token=token_if_civitai_url)
# Validate loRA file extension
valid_extensions = {'.safetensors', '.ckpt', '.pt', '.pth', '.sft'}
if lora:
    if not any(lora.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora}")
        lora = None
    else:
        clear_output()
        print("loRA downloaded succesfully!")

# model = model_download(qwen_model_download_url, "/content/ComfyUI/models/unet")
# if also_download_qwen_image_edit:
model_edit = model_download(qwen_edit_model_download_url, "/content/ComfyUI/models/unet")
text_encoder = model_download(qwen_text_encoder_download_url, "/content/ComfyUI/models/clip")
vae = model_download(qwen_vae_download_url, "/content/ComfyUI/models/vae")
# controlnet = model_download(qwen_instantX_controlnet_download_url, "/content/ComfyUI/models/controlnet")
turbo = model_download(qwen_speedup_lora_download_url, "/content/ComfyUI/models/loras")



use_cloudflare = False # @param {type:"boolean"}
use_interface_in_cell = False # @param {type:"boolean"}


if use_cloudflare:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb

    import subprocess
    import threading
    import time
    import socket
    import urllib.request

    def iframe_thread(port):
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', port))
          if result == 0:
            break
          sock.close()
      print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

      p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
          print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
        #print(l, end='')
    clear_output()

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if using_T4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server


elif use_interface_in_cell:
    import threading
    import time
    import socket
    def iframe_thread(port):
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', port))
          if result == 0:
            break
          sock.close()
      from google.colab import output
      output.serve_kernel_port_as_iframe(port, height=1024)
      clear_output()
      print("to open it in a window you can open this link here:")
      output.serve_kernel_port_as_window(port)

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if using_T4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server

else:
    import socket, time, threading
    from google.colab import output

    def link_thread(port):
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                break
            sock.close()
        clear_output()
        print("Click the link below to launch the comfyui interface")
        output.serve_kernel_port_as_window(port)


    # Start thread for port 8188
    threading.Thread(target=link_thread, daemon=True, args=(8188,)).start()

    if using_T4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server






